In [255]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle

from datetime import timedelta

import matplotlib.pyplot as plt

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [256]:
dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

# print(os.environ["db_name"])

# conn = pymysql.connect(
#   host = os.environ['db_host'],
#   port = int(os.environ['db_port']),
#   user = os.environ['db_user'],
#   password = os.environ['db_password'],
#   db = os.environ['db_name']
#   # charset= os.environ['db_charset']
# )

True

In [257]:
# curs = conn.cursor()

# sql = "SELECT cpg, DATE_FORMAT(save_time_id, '%Y-%m-%d'), HOUR(save_time) FROM gsmon_solar_data WHERE rtu_id='24' AND lerr != '57'"
# curs.execute(sql)

# result = curs.fetchall()

# print(result)

In [258]:
# with open('plant_data', 'wb') as file :
#   pickle.dump(result, file)

In [259]:
# with open( "plant_data", "rb" ) as file:
#     data = pickle.load(file)

In [260]:
plant_data = pd.read_pickle('plant_data')

data = pd.DataFrame(plant_data, columns=['누적발전량', '날짜', '시간'])

data.tail()

,누적발전량,날짜,시간
920524,238822000,2023-06-28,19
920525,238822000,2023-06-28,19
920526,238822000,2023-06-28,19
920527,238822000,2023-06-28,19
920528,238822000,2023-06-28,19


In [261]:
# 날짜 분리하기
data['날짜'] = pd.to_datetime(data['날짜'])

data['년'] = data['날짜'].dt.year
data['월'] = data['날짜'].dt.month
data['일'] = data['날짜'].dt.day

data = data[['날짜', '년','월','일','시간','누적발전량']]

year = [2020, 2023]

filter_y = data['년'].isin(year)
pg_data = data.drop(index=data[filter_y].index, axis=0)

pg_data.tail()

,날짜,년,월,일,시간,누적발전량
796473,2022-12-31,2022,12,31,17,204302000
796474,2022-12-31,2022,12,31,17,204302000
796475,2022-12-31,2022,12,31,17,204302000
796476,2022-12-31,2022,12,31,17,204302000
796477,2022-12-31,2022,12,31,17,204302000


In [262]:
# 2021년 1월 1일의 전일 데이터 처리를 위한 2020년 12월 31일 마지막 row 가져오기

filter_l = data['년']==2020

last_data_2020 = data[filter_l]

last_data = (last_data_2020.iloc[-1, :]).to_frame()

last_data= last_data.transpose()

print(last_data.shape)
print(pg_data.shape)

pg_data = pd.concat([last_data, pg_data], axis=0)

pg_data.head()

(1, 6)
(532266, 6)


,날짜,년,월,일,시간,누적발전량
264211,2020-12-31,2020,12,31,17,72059000
264212,2021-01-01,2021,1,1,7,72059000
264213,2021-01-01,2021,1,1,7,72059000
264214,2021-01-01,2021,1,1,7,72059000
264215,2021-01-01,2021,1,1,7,72059000


In [263]:
# 전일 최대 누적 발전량 column 추가

# pg_data = pg_data.reset_index(drop=True)

# for i in range(1, len(pg_data)) :
#   yesterday_date = pg_data.iloc[i]['날짜']-timedelta(days=1)
#   # print(yesterday_date)
#   filter_d = pg_data['날짜'] == yesterday_date.strftime('%Y-%m-%d')
#   pg_data.loc[i, '전일최대누적발전량'] = max(pg_data[filter_d]['누적발전량'])


# pg_data.loc[0]['누적발전량']

# for문 적용 테스트
# yesterday_date = pg_data.iloc[1]['날짜']-timedelta(days=1)
# filter_d = pg_data['날짜'] == yesterday_date.strftime('%Y-%m-%d')
# pg_data.loc[2, '전일최대누적발전량'] = max(pg_data[filter_d]['누적발전량'])
# print(pg_data)
# print(yesterday_date.strftime('%Y-%m-%d'))

In [264]:
# pg_data = pg_data.iloc[1:, :]
# print(pg_data.tail())

# with open('plant_data_addy', 'wb') as file :
#   pickle.dump(pg_data, file)

In [265]:
with open('plant_data_addy', 'rb') as file :
  data = pickle.load(file)
  
# print(data)


# 금일 누적 발전량
data['금일누적발전량'] = data['누적발전량'] - data['전일최대누적발전량']

print(data.tail())
# 샘플 데이터 (금일 누적 데이터 확인)
# filter_test = data['날짜'] == '2022-12-31'
# fig, axe = plt.subplots(figsize=(12, 8), layout='constrained')
# axe.bar(data[filter_test]['시간'], data[filter_test]['금일누적발전량'], color='g', label='금일누적발전량')

               날짜     년   월   일  시간      누적발전량    전일최대누적발전량   금일누적발전량
532262 2022-12-31  2022  12  31  17  204302000  204106000.0  196000.0
532263 2022-12-31  2022  12  31  17  204302000  204106000.0  196000.0
532264 2022-12-31  2022  12  31  17  204302000  204106000.0  196000.0
532265 2022-12-31  2022  12  31  17  204302000  204106000.0  196000.0
532266 2022-12-31  2022  12  31  17  204302000  204106000.0  196000.0


In [266]:
# 시간대별 누적발전량 산출 & 시간대별 발전량 산출

dg = data.groupby(["년","월","일","시간"])['금일누적발전량'].max()

dg = dg.reset_index()

for i in range(1, len(dg)) :
  if dg.loc[i, '금일누적발전량'] - dg.loc[i-1, '금일누적발전량'] < 0 :
    dg.loc[i, '금일발전량'] = 0
  else :
    dg.loc[i, '금일발전량'] = dg.loc[i, '금일누적발전량'] - dg.loc[i-1, '금일누적발전량']

data = dg.fillna(0.0)
  
# print(data)

# 샘플 데이터 (금일 누적 데이터 확인)
# filter_test = (data['년'] == 2022) & (data['월'] == 12) & (data['일'] == 29)

# print(data[filter_test])

# fig, axe = plt.subplots(figsize=(12, 8), layout='constrained')
# axe.bar(data[filter_test]['시간'], data[filter_test]['금일발전량'], color='g', label='금일발전량')

# print(data[filter_test]['금일발전량'].sum())  # 실데이터 비교 결과 일치


data = data[['년', '월', '일', '시간', '금일발전량']]

print(data.tail())

         년   월   일  시간    금일발전량
9667  2022  12  31  13  34000.0
9668  2022  12  31  14  28000.0
9669  2022  12  31  15  17000.0
9670  2022  12  31  16   5000.0
9671  2022  12  31  17   1000.0


In [268]:
data

,년,월,일,시간,금일발전량
0,2021,1,1,7,0.0
1,2021,1,1,8,6000.0
2,2021,1,1,9,16000.0
3,2021,1,1,10,24000.0
4,2021,1,1,11,30000.0
...,...,...,...,...,...
9667,2022,12,31,13,34000.0
9668,2022,12,31,14,28000.0
9669,2022,12,31,15,17000.0
9670,2022,12,31,16,5000.0
